In [1]:
import pandas as pd
import tensorflow as tf
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import keras
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import merge
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Add
from keras.layers import Activation, BatchNormalization
# regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers import LSTM, Bidirectional
from keras.utils import plot_model
from keras.layers import Dropout
from scipy import signal
import openpyxl

class LossHistory(keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))

def create_dataset(dataset, timesteps):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps - 1):
        a = dataset[i:(i + timesteps), 0]
        dataX.append(a)
        dataY.append(dataset[i + timesteps, 0])
    return np.array(dataX), np.array(dataY)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

data_path = 'C:/Users/Smartgrid_ML/Desktop/KMS/ercot_2016.xlsx'
data_source = pd.read_excel(data_path, sheet_name='numbers')
data_source = pd.DataFrame(data_source)
data_set = data_source.iloc[:, 3]
wind_2016 = np.array(data_set)
wind_2016 = wind_2016.reshape(-1, 1)
installed_2016 = data_source.iloc[:, 4]
installed_2016 = np.array(installed_2016)
installed_2016 = installed_2016.reshape(-1, 1)

data_path = 'C:/Users/Smartgrid_ML/Desktop/KMS/ercot_2017.xlsx'
data_source = pd.read_excel(data_path, sheet_name='numbers')
data_source = pd.DataFrame(data_source)
data_set = data_source.iloc[:, 3]
wind_2017 = np.array(data_set)
wind_2017 = wind_2017.reshape(-1, 1)
installed_2017 = data_source.iloc[:, 4]
installed_2017 = np.array(installed_2017)
installed_2017 = installed_2017.reshape(-1, 1)

data_path = 'C:/Users/Smartgrid_ML/Desktop/KMS/ercot_2018.xlsx'
data_source = pd.read_excel(data_path, sheet_name='numbers')
data_source = pd.DataFrame(data_source)
data_set = data_source.iloc[:, 3]
wind_2018 = np.array(data_set)
wind_2018 = wind_2018.reshape(-1, 1)
installed_2018 = data_source.iloc[:, 4]
installed_2018 = np.array(installed_2018)
installed_2018 = installed_2018.reshape(-1, 1)

wind = np.concatenate((wind_2016, wind_2017, wind_2018), axis=None)
wind = wind.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
wind = scaler.fit_transform(wind)
Maxvalue_1 = scaler.data_max_
print(wind.shape)

np.savetxt('ERCOT_WIND_DATA.txt', wind)

Training_set = wind[0:20000, :]
Validation_set = wind[20001:24304, :]
Test_set = wind[24305:26305:, :]

Tr_mean = np.nanmean(wind)
Tr_var = np.nanvar(wind)

print('Dataset mean = ', Tr_mean, 'var = ', Tr_var)
print('Maximum value = ', Maxvalue_1)

features = 1
timesteps = 24
trainX, trainY = create_dataset(Training_set, timesteps)
validX, validY = create_dataset(Validation_set, timesteps)
testX, testY = create_dataset(Test_set, timesteps)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
validX = np.reshape(validX, (validX.shape[0], validX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# -----------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------
import keras.backend.tensorflow_backend as K
with K.tf.device('/gpu:0'):

    visible1 = Input(shape=(timesteps, features))
    con10 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(visible1)
    con10 = Activation('relu')(con10)
    
    bi11 = (Bidirectional(LSTM(16, return_sequences=True)))(visible1) 
    bi12 = (Bidirectional(LSTM(16, return_sequences=True)))(bi11)
    con11 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(bi12)
    con11 = Activation('relu')(con11)
    res13 = keras.layers.Concatenate()([bi12, con11, visible1])
    con20 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(res13)
    con20 = Activation('relu')(con20)

    bi13 = (Bidirectional(LSTM(16, return_sequences=True)))(res13)
    bi14 = (Bidirectional(LSTM(16, return_sequences=True)))(bi13)
    con12 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(bi14)
    con12 = Activation('relu')(con12)
    res14 = keras.layers.Concatenate()([res13, con20, bi14, con12, visible1])
    con21 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(res14) 
    con21 = Activation('relu')(con21)

    bi15 = (Bidirectional(LSTM(16, return_sequences=True)))(res14)
    bi16 = (Bidirectional(LSTM(16, return_sequences=True)))(bi15)
    con13 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(bi16)
    con13 = Activation('relu')(con13)
    res15 = keras.layers.Concatenate()([res14, con21, bi16, con13, visible1])
    con22 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(res15)
    con22 = Activation('relu')(con22)

    bi17 = (Bidirectional(LSTM(16, return_sequences=True)))(res15)
    bi18 = (Bidirectional(LSTM(16, return_sequences=True)))(bi17)
    con14 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(bi18)
    con14 = Activation('relu')(con14)
    res16 = keras.layers.Concatenate()([res15, con22, bi18, con14, visible1])
    con23 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(res16)
    con23 = Activation('relu')(con23)

    bi19 = (Bidirectional(LSTM(16, return_sequences=True)))(res16)
    bi20 = (Bidirectional(LSTM(16, return_sequences=True)))(bi19)
    con15 = Conv1D(filters=32, kernel_size=1, strides=1, padding='same')(bi20)
    con15 = Activation('relu')(con15)
    res17 = keras.layers.Concatenate()([res16, con23, bi20, con15, visible1])

    bi21 = (Bidirectional(LSTM(32)))(res17)

    dense11 = Dense(24, activation='relu')(bi21)
    out1 = Dense(1, activation='relu')(dense11)

    model = Model(inputs=[visible1], outputs=[out1])  # multi-input, multi-output

#########################################

print('Shape of trainX = ', trainX.shape)
print(model.summary())
#plot_model(model, show_shapes=True, to_file=title+'model_img.png')

reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1)  # 에포크 1동안 좋아지지 않으면 호출

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'mape'])

batch_size = 40
epochs = 20

history = LossHistory()
history.init()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(validX, validY), callbacks=[history, reducelr])  # , checkpoint

trainPredict = model.predict(trainX, batch_size=batch_size)
testPredict = model.predict(testX, batch_size=batch_size)

trainPredict = trainPredict.reshape([-1])
testPredict = testPredict.reshape([-1])
testY = testY.reshape([-1])
trainY = trainY.reshape([-1])


np.savetxt('trainPredict_5.txt', trainPredict)
np.savetxt('testPredict_5.txt', testPredict)
np.savetxt('testY_5.txt', testY)
np.savetxt('trainY_5.txt', trainY)

print('--------------------------------------------------------')
print(' history.mses(=loss) = ', history.mses)  # loss == mse
print(' history.maes        = ', history.maes)
print(' history.mapes        = ', history.mapes)
print(' history.val_mses(=loss)= ', history.val_mses)  # loss == mse
print(' history.val_maes       = ', history.val_maes)
print(' history.val_mapes       = ', history.val_mapes)
print('--------------------------------------------------------')

plt.figure(1)
plt.plot(testPredict, 'r')
plt.plot(testY, 'b')
plt.show()

plt.figure(2)
plt.plot(history.mses, 'b--')
plt.plot(history.val_mses, 'b:')
plt.plot(history.maes, 'ro')
plt.plot(history.val_maes, 'r+')
plt.ylabel('Loss and Accuracy')
plt.xlabel('epoch')
plt.show()

print('Test Score > MSE == ', mean_squared_error(testY, testPredict), ' MAE == ', mean_absolute_error(testY, testPredict), 'MAPE == ', mean_absolute_percentage_error(testY, testPredict))

Using TensorFlow backend.
W0303 16:33:11.492074 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0303 16:33:11.492074 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0303 16:33:11.495074 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



(26311, 1)
Dataset mean =  0.36639967284314107 var =  0.04237001272906482
Maximum value =  [19099.07421875]


W0303 16:33:14.780826 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Shape of trainX =  (19975, 24, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24, 1)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 24, 32)       2304        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 24, 32)       6272        bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 24, 32)       1056        bidirectional_2[0][0]            
___________________________________________________________________________

W0303 16:33:15.214932 12196 deprecation.py:323] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0303 16:33:24.376258 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0303 16:33:25.159436 12196 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 19975 samples, validate on 4278 samples
Epoch 1/20
19975/19975 [==============================] - 327s 16ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - mean_absolute_error: 0.0396 - mean_absolute_percentage_error: 43724.3571 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - val_mean_absolute_error: 0.0304 - val_mean_absolute_percentage_error: 13.2126
Epoch 2/20
19975/19975 [==============================] - 313s 16ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0272 - mean_absolute_percentage_error: 49504.8881 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - val_mean_absolute_error: 0.0274 - val_mean_absolute_percentage_error: 10.0435
Epoch 3/20
19975/19975 [==============================] - 313s 16ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - mean_absolute_error: 0.0261 - mean_absolute_percentage_error: 47880.6319 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - val_mean_absolute_error: 0.0267 - val_mean_absolute_perce

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Test Score > MSE ==  0.0011000575694189683  MAE ==  0.024124829549418088 MAPE ==  9.585872931814343
